GENERAL STEPS FOR NN FUNCTIONING:
* Load a net model and send to device. Load their weights if we have an already trained model.
* TRAIN:
    * Choose loss function with their parameters. Load/compute weights if needed.
    * Choose optimizer and its parameters.
    * Load the data to train.
    * Iterate per epoch and train:
        * Set net to train and choose the metrics (goodness indicators)
        * Iterate per batch in the loader, send batch to device, zero grad the optimizer, forward the data through the net, evaluate loss, back propagation for the loss and gradient descent for the optimizer (update one step).
        * Save per epoch the loss and the metrics (I'm not doing the latter in this notebook)
    * Save net configuration if validation loss has descended.
* Evaluate: repeat the process with evaluation of the model, and a softmax to decide the dominant class.

In [14]:
import sys
import copy
import random
import itertools
import numpy   as np
import pandas  as pd
import os.path as osp
from   glob import glob

import invisible_cities.io.dst_io as dio

import torch
from   torch_geometric.data import Data, Dataset
from   torch_geometric.data.makedirs import makedirs

In [2]:
def edge_index(event, max_distance = np.sqrt(3), coord_names = ['xbin', 'ybin', 'zbin'], directed = False):
    ''' 
    Creates the edge index tensor, with shape [2, E] where E is the number of edges.
    It contains the index of the nodes that are connected by an edge. 
    '''
    coord = event[coord_names].T
    edges = []
    if directed: node_comb = itertools.combinations(coord, 2)
    else: node_comb = itertools.permutations(coord, 2)
    for i, j in node_comb:
        dis = np.linalg.norm(coord[i].values - coord[j].values)
        if dis <= max_distance:
            edges.append([i, j])
    edges = torch.tensor(edges).T
    return edges

In [34]:
def graphData(event, features = ['energy'], label_n = ['segclass'], max_distance = np.sqrt(3), coord_names = ['xbin', 'ybin', 'zbin'], directed = False, simplify_segclass = False):
    edges = edge_index(event, max_distance=max_distance, coord_names=coord_names, directed=directed)
    #nodes features, for now just the energy; the node itself is defined by its position
    nodes = torch.tensor(event[features].values)
    #nodes segmentation label
    seg = event[label_n].values
    if simplify_segclass:
        label_map = {1:1, 2:2, 3:3, 4:1, 5:2, 6:3, 7:4}
        seg = np.array([label_map[i] for i in seg])
    #we can try to add also the transformation just to have track + blob (+ ghost)
    label = torch.tensor(seg)
    #maybe add binclass label?
    graph_data = Data(edge_index = edges, x = nodes, y = label, num_nodes = len(nodes))
    return graph_data

In [35]:
def graphDataset(file, features = ['energy'], label_n = ['segclass'], max_distance = np.sqrt(3), coord_names = ['xbin', 'ybin', 'zbin'], directed = False, simplify_segclass = False):
    df = dio.load_dst(file, 'DATASET', 'BeershebaVoxels')
    dataset = []
    for dat_id, event in df.groupby('dataset_id'):
        event = event.reset_index(drop = True)
        graph_data = graphData(event, features=features, label_n=label_n, max_distance=max_distance, coord_names=coord_names, directed = directed, simplify_segclass = simplify_segclass)
        dataset.append(graph_data)
    return dataset

In [36]:
class Dataset(Dataset):
    def __init__(self, root, tag = '0nubb', transform=None, pre_transform=None, pre_filter=None, directed = False, simplify_segclass = False):
        self.sort = lambda x: int(x.split('_')[-2])
        self.tag = tag
        self.directed = directed
        self.simplify_segclass = simplify_segclass
        super().__init__(root, transform, pre_transform, pre_filter)
        
    @property
    def raw_file_names(self):
        ''' 
        Returns a list of the raw files in order (supossing they are beersheba labelled files that have the structure beersheba_label_N_tag.h5)
        '''
        rfiles = [i.split('/')[-1] for i in glob(self.raw_dir + '/*_{}.h5'.format(self.tag))]
        return sorted(rfiles, key = self.sort)

    @property
    def processed_file_names(self):
        '''
        Returns a list of the processed files in order (supossing they are stored tensors with the structure data_N.pt)
        '''
        pfiles = [i.split('/')[-1] for i in glob(self.processed_dir + '/data_*_{}.pt'.format(self.tag))]
        return sorted(pfiles, key = self.sort)
    
    def process(self):
        makedirs(self.processed_dir)
        already_processed = [self.sort(i) for i in self.processed_file_names]
        for raw_path in self.raw_paths:
            idx = self.sort(raw_path)
            if np.isin(idx, already_processed):
                #to avoid processing already processed files
                continue
            data = graphDataset(raw_path, directed=self.directed, simplify_segclass=self.simplify_segclass)

            #if self.pre_filter is not None and not self.pre_filter(data):
            #    continue

            #if self.pre_transform is not None:
            #    data = self.pre_transform(data)

            torch.save(data, osp.join(self.processed_dir, f'data_{idx}_{self.tag}.pt'))
        

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}_{self.tag}.pt'))
        return data

    def join(self):
        #print('Joining ', self.processed_file_names)
        dataset = []
        for processed_path in self.processed_paths:
            dataset += torch.load(processed_path)
        return dataset

In [6]:
from torch_geometric.nn import GCNConv
from torch.nn import BatchNorm1d, CrossEntropyLoss
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

Structure of the mock GNN

![Alt text](mock_gcn.png)

In [7]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = torch.nn.ModuleList()

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList()

        # The log softmax layer
        self.softmax = None

        ############ BUILDING THE LAYERS ##############
        ## 1. Following the image, there are GCNConv num_layers and BatchNorm1d num_layers - 1
        ## 2. The GCNConv will have input_dim as the num of node features (1 for now, the energy)
        ## 3. The hidden_dim is the dimension in the inner channels, in here it's kept constant but
        ##    in a future could be changed during the process
        ## 4. The output_dim should be equal to the number of classes we can have per node
        ## 5. The output dim of a GCNConv is the input dim for the following

        self.convs.append(GCNConv(input_dim, hidden_dim))
        self.bns.append(BatchNorm1d(hidden_dim))
        for i in range(num_layers - 1): #take one because the first layer was already written outside
          self.convs.append(GCNConv(hidden_dim, hidden_dim))
          self.bns.append(BatchNorm1d(hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))
        #no bns append here as the last GCNConv is followed by the softmax

        #softmax is applied in the 1 dimension (to each node individually)
        self.softmax = torch.nn.LogSoftmax(dim = 1)

        # Probability of an element getting zeroed
        self.dropout = torch.nn.Dropout(dropout)

        # Applies the rectified linear unit function (keeps only positive values)
        self.relu = torch.nn.ReLU()

        #########################################

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index):
        out = None

        ############# PASS THE VECTORS ############
        ## 1. We need now to pass the vectors through the layers
        ## 2. If return_embeds is True, then skip the last softmax layer
        for conv, bn in zip(self.convs, self.bns):
          x = conv(x, edge_index)
          x = bn(x)
          x = self.relu(self.dropout(x))
        x = self.convs[-1](x, edge_index)
        if self.return_embeds: out = x
        else: out = self.softmax(x)
        #########################################

        return out

In [8]:
def train(model, loader, device, optimizer, loss_fn):
    # Tell the model it's going to train
    model.train()
    loss = 0

    # Iterate for the batches in the data loader
    for batch in loader:
        # Pass the batch to device (cuda)
        batch = batch.to(device)

        # Zero grad the optimizer
        optimizer.zero_grad()

        # Pass the data to the model
        out = model.forward(batch.x.type(torch.float), batch.edge_index) 

        # Now we pass the output and the labels to the loss function
        # We will use nll_loss (negative log likelihood, useful to train C classes bc we can add weights for each class)
        # This loss will need input (N, C) target (N); being C = num of classes, N = batch size
        
        # We read the label, transform into long tensor (needed by this loss function), pass to cuda device and shifted by one 
        # because for the output the classes are from [0, 6] and for the labels they are [1, 7]
        label = batch.y.type(torch.LongTensor).to(device) - 1

        # The reshape is needed to pass from a (N, 1) shape (automatically appears when doing
        # batch.y), to a (N) shape as we need; the output of the net is already (N, C) if it's properly built
        loss = loss_fn(out, torch.reshape(label, (-1,)))
        
        # Back propagation (compute gradients of the loss with respect to the weights in the model)
        loss.backward()
        # Gradient descent (update the optimizer)
        optimizer.step()
    return loss.item()

In [17]:
def accuracy(pred, true):
    acc = sum(pred == true) / len(pred)
    return acc

In [15]:
def IoU(pred, true, nclass = 3):
    """
        Intersection over union is a metric for semantic segmentation.
        It returns a IoU value for each class of our input tensors/arrays.
    """
    eps = sys.float_info.epsilon
    confusion_matrix = np.zeros((nclass, nclass))

    for i in range(len(true)):
        confusion_matrix[true[i]][pred[i]] += 1

    IoU = []
    for i in range(nclass):
        IoU.append((confusion_matrix[i, i] + eps) / (sum(confusion_matrix[:, i]) + sum(confusion_matrix[i, :]) - confusion_matrix[i, i] + eps))
    return np.array(IoU)

In [16]:
def eval(model, loader, device):
    # Set the model to evaluate
    model.eval()
    y_true = []
    y_pred = []
    # Iterate for the batches in the data loader
    for batch in loader:
        # Put batch into device (cuda)
        batch = batch.to(device)

        # Pass the data to the model
        with torch.no_grad():
            out = model.forward(batch.x.type(torch.float), batch.edge_index)
        
        # For each node set the maximum argument to pick a class
        pred = out.argmax(dim=-1, keepdim=True)  

        #Once again, the labels are shifted by 1 to match the prediction positions (explained in train fun)
        true = torch.reshape(batch.y, (-1,)).detach().cpu() - 1
        
        #Append the results to lists
        y_pred.append(torch.reshape(pred, (-1,)).detach().cpu())
        y_true.append(true)
    
    #Concatenate the items in the list and transform into array
    y_pred = torch.cat(y_pred).numpy()
    y_true = torch.cat(y_true).numpy()

    #Identify the neighbor segclass with their original segclass to compare each node
    label_map = {0:0, 1:1, 2:2, 3:0, 4:1, 5:2, 6:3}
    y_pred = np.array([label_map[i] for i in y_pred])
    y_true = np.array([label_map[i] for i in y_true])
    
    # Compare and return an accuracy (number of success nodes / all nodes)
    # Not the best, it is better the IoU for segmentation
    acc = accuracy(y_pred, y_true)
    iou = IoU(y_pred, y_true, nclass=4)
    return acc, iou, y_pred, y_true

In [18]:
def create_idx_split(dataset, train_perc):
    indices = np.arange(len(dataset))
    valid_perc = (1 - train_perc) / 2
    random.shuffle(indices)
    train_data = torch.tensor(np.sort(indices[:int((len(indices)+1)*train_perc)])) #Remaining 80% to training set
    valid_data = torch.tensor(np.sort(indices[int((len(indices)+1)*train_perc):int((len(indices)+1)*(train_perc + valid_perc))]))
    test_data = torch.tensor(np.sort(indices[int((len(indices)+1)*(train_perc + valid_perc)):]))
    idx_split = {'train':train_data, 'valid':valid_data, 'test':test_data}
    return idx_split


Select the device

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

Device: cuda


Define the parameters

In [20]:
# Arguments for the net and the train
args = {
      'device': device,
      'nclass':7,
      'num_layers': 3,
      'hidden_dim': 30,
      'dropout': 0.3,
      'lr': 0.001,
      'epochs': 100,
      'batch_size': 50
  }

Load the train/validation/test sets of events

In [23]:
# Creation of the dataset, index split and data loaders for each case
file_path = '/mnt/lustre/scratch/nlsas/home/usc/ie/mpm/NEXT100/labelled_data/0nubb/554mm_voxels/'

dataset = Dataset(file_path, '0nubb').join()
idx_split = create_idx_split(dataset, 0.8)

train_loader = DataLoader([dataset[i] for i in idx_split['train']], batch_size=args['batch_size'], shuffle=True, num_workers=0)
valid_loader = DataLoader([dataset[i] for i in idx_split['valid']], batch_size=args['batch_size'], shuffle=False, num_workers=0)
test_loader = DataLoader([dataset[i] for i in idx_split['test']], batch_size=args['batch_size'], shuffle=False, num_workers=0)

Compute the weight loss per class

In [24]:
def weight_loss(file_names, correct = False):
    #correct assigns to the ghost class the desired inverse freq and redistributes the rest
    seg = pd.Series(dtype='int')
    for f in file_names:
        seg = seg.append(dio.load_dst(f, 'DATASET', 'BeershebaVoxels').segclass)
    freq = np.bincount(seg - 1, minlength=max(seg))
    inv_freq = 1. / freq
    inv_freq = inv_freq / sum(inv_freq)
    if correct:
        redistr = inv_freq[:-1] * (1 - correct) / sum(inv_freq[:-1])
        inv_freq = np.append(redistr, correct)
    return inv_freq
    

In [25]:
files_for_weights = glob(file_path + 'raw/*.h5')
inv_freq = weight_loss(files_for_weights, correct = 0.1)

Load the model

In [26]:
# Initiate the model with the previous args and set to device
model = GCN(dataset[0].num_features, args['hidden_dim'],
              args['nclass'], args['num_layers'],
              args['dropout']).to(device)

Start training

In [27]:
# Set true if we want to train in the next cell
start_train = True

In [28]:
if start_train:
  # Start from zero the model (not using a trained model)
  model.reset_parameters()

  # Initiate the optimizer with the model parameters and a learning rate
  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

  # Pick the loss function
  loss_fn = torch.nn.NLLLoss(weight=torch.Tensor(inv_freq).to(device)) #CrossEntropyLoss(weight=torch.Tensor(inv_freq).to(device)) #

  best_model = None
  best_valid_acc = 0

  # Iterate on the number of epochs
  pred = []
  true = []
  for epoch in range(1, 1 + args["epochs"]):
    # Train the model with the fucntion
    print('Training...')
    loss = train(model, train_loader, device, optimizer, loss_fn)

    #Evaluate the model with the function and for the 3 sets of data
    print('Evaluating...')
    train_acc, train_iou, _, _ = eval(model, train_loader, device)
    val_acc, val_iou, y_pred, y_true = eval(model, valid_loader, device)
    test_acc, test_iou, _, _ = eval(model, test_loader, device)
    #pred.append(y_pred)
    #true.append(y_true)
    # Store the model if the validation accuracy improved
    if val_acc > best_valid_acc:
        best_valid_acc = val_acc
        best_model = copy.deepcopy(model)
    # Print the important variables for epoch
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * val_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')
    print(f'Blob IoU train: {100 * train_iou[-2]:.2f}%, '
          f'Blob IoU valid: {100 * val_iou[-2]:.2f}%, '
          f'Blob IoU test: {100 * test_iou[-2]:.2f}%, ')

Training...
Evaluating...
Epoch: 01, Loss: 2.1279, Train: 77.36%, Valid: 76.66% Test: 77.22%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 02, Loss: 2.0201, Train: 77.36%, Valid: 76.66% Test: 77.22%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 03, Loss: 1.9473, Train: 77.36%, Valid: 76.66% Test: 77.22%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 04, Loss: 1.9292, Train: 8.28%, Valid: 7.52% Test: 9.20%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 05, Loss: 1.8599, Train: 4.38%, Valid: 4.14% Test: 5.65%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 06, Loss: 1.8659, Train: 4.38%, Valid: 4.14% Test: 5.65%
Blob IoU train: 0.00%, Blob IoU valid: 0.00%, Blob IoU test: 0.00%, 
Training...
Evaluating...
Epoch: 07, 

In [32]:
train_acc, train_iou, _, _ = eval(best_model, train_loader, device)
valid_acc, valid_iou, _, _ = eval(best_model, valid_loader, device)
test_acc, test_iou, _, _  = eval(best_model, test_loader, device)

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')
print(f'Best model IoU blob: '
      f'Train: {100 * train_iou[-2]:.2f}%, '
      f'Valid: {100 * valid_iou[-2]:.2f}% '
      f'Test: {100 * test_iou[-2]:.2f}%')

Best model: Train: 77.08%, Valid: 77.72% Test: 76.36%
Best model IoU blob: Train: 6.44%, Valid: 9.81% Test: 4.81%


It could work better, but I guess there are several factors to take into account for which this is failing:
* DISCONNECTED TRACKS! We still have this problem, maybe we can try bigger voxelization if this works
* Maybe TOO COMPLEX GRAPHS (lots of nodes and edges) without much information... Bigger voxels or clouds JA analysis could help with that
* NOT CLEAN DATA, lots of ghosts etc that can be easily cleaned (although with the disconnected tracks becomes more dangerous)
* NOT EVENT INFORMATION; I'm passing graphs to the net without any information of the event they are in. For this, it should take into account the batch information. Search information on that, to see if there are any layers for the graphs that involve this info.
* NET ARCHITECTURE: not the optimal, since there are things that maybe don't match with what we need (for example the previous bullet)
* NET PARAMETERS: there are some params that we could play with
* LOSS FUNCTION: not used yet the weights, and don't know either if this is the optimal one; could try focal loss or something as we used with the regular segmentation
* TRAIN MORE THOUGHTFULLY; not all the data is being used...
* BATCHING; Play also with the batching, maybe batching 1 by 1 makes the net aware of each individual event...
* 1 TRACK CUT WOULD SOLVE EVENT-GRAPH RELATION; for now it's not compatible witht the track disconnection.
* TRY CLASSIFICATION NET; need to change this a bit and think how to unify the thing graph - event, or how to label each graph instead of each event...
* TRY SIMPLIFYING THE INPUT CLASSES; instead of reducing them after passing the DNN, process data to already have it simplified


In [24]:
_, y_pred, y_true = eval(best_model, valid_loader, device)


In [25]:
np.histogram(y_pred, max(np.unique(y_pred)) + 1, range = (min(np.unique(y_pred)), max(np.unique(y_pred)) + 1))

(array([ 302, 8018,  955]), array([0., 1., 2., 3.]))

In [26]:
np.histogram(y_true, max(np.unique(y_true)) + 1, range = (min(np.unique(y_true)), max(np.unique(y_true)) + 1))

(array([ 311, 7290, 1671,    3]), array([0., 1., 2., 3., 4.]))

Unbalance between other and track class... Blob class seems to be right!! So maybe we must try no differences between track and other...

I hope this method is only adding and deciding by the sum of the energy etc... Should try other architectures